# Prepare the datasets

## data449_text.csv

We use the data set data449_cats which is directly extracted from Label Studio with the csv format. It is made out of the labellisation of data. Our objective is to reduce it to two columns with the text and 0 labels.

In [11]:
import pandas as pd 

# Read in the data
df = pd.read_csv('../data/raw/data449_cats.csv')

# Drop columns
df = df[["text"]].copy()

# Save to csv
df.to_csv('../data/processed/data449_text.csv', index=False)

## long_short_texts.csv

In [51]:
import pandas as pd
import os
import glob

We import the shorten texts

In [53]:
# Read in the data
df_short_text = pd.read_excel(r'../data/raw/short_texts.xlsx').loc[0:495, ["code","entreprise","text"]]

We build the long text database

In [59]:
# Build a database of long text
df_long_text = pd.DataFrame(columns=['code','text'])

# Loop through the long text
texts = glob.glob(r"../data/text/txt/*.txt")

list_of_codes = []
list_of_texts = []

for text in texts:
    filename = os.path.basename(text).split('.')[0][0:12]
    list_of_codes.append(filename)
    with open(text, 'r', encoding="utf-8") as f:
        list_of_texts.append(f.read())

# Create a new DataFrame
df_new_data = pd.DataFrame({'code': list_of_codes, 'text': list_of_texts})

# Reset the columns of df_new_data to only include 'code' and 'text'
df_new_data = df_new_data[['code', 'text']]

# Concatenate the new DataFrame with df_long_text
df_long_text = pd.concat([df_long_text, df_new_data], axis=0, ignore_index=True)

Retrieve the name of the company

In [103]:
# retrieve the name of compagnies
df_entreprise = pd.read_excel(r"../data/raw/full_data_link_legifrance.xlsx")
df_entreprise = df_entreprise[['Fichier', 'Entreprise']].copy()
# df_entreprise = df_entreprise.rename(columns={'Entreprise': 'entreprise'})
df_entreprise = df_entreprise.rename(columns={'Fichier': 'code'})

df_entreprise['code'] = df_entreprise['code'].str.slice(101, 113)

# Merge the two dataframes
df_long_text = pd.merge(df_long_text, df_entreprise, how='left', left_on='code', right_on='code')

df_long_text = df_long_text[['code', 'entreprise', 'text']]

Merging the datasets

In [104]:
# We merge the datasets
df_text = pd.merge(df_short_text, df_long_text, how='inner', suffixes=('_short', '_long'), on='code')

In [111]:
df_text = df_text[['code', 'entreprise_short', 'text_short', 'text_long']].copy()
df_text = df_text.rename(columns={'entreprise_short': 'entreprise'})

In [112]:
df_text.head()
df_text.to_csv('../data/processed/short_long_texts.csv', index=False)